## 6-7. 프로젝트: 멋진 작사가 만들기
1) Step 1. 데이터 다운로드  
Song Lyrics 데이터를 다운  
  
2) Step 2. 데이터 읽어오기  
  
3) Step 3. 데이터 정제  
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!  
preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.  
  
추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권합니다.  
  
4) Step 4. 평가 데이터셋 분리  
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!  
```
enc_train, enc_val, dec_train, dec_val = <코드 작성>

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


out:

Source Train: (124960, 14)
Target Train: (124960, 14)
```

  
5) Step 5. 인공지능 만들기  
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)  
``` 
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
    
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

```

***

평가문항
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  
텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?  
  
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  
특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?  
  
3. 텍스트 생성모델이 안정적으로 학습되었는가?   
텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?   



## 1) Step 1. 데이터 다운로드  
Song Lyrics 데이터를 다운  
  
## 2) Step 2. 데이터 읽어오기  

In [1]:
import glob # 파일을 읽어오는 작업을 하기가 아주 용이
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' # * = all : 모두 읽는다.

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['How does a bastard, orphan, son of a whore', 'And a Scotsman, dropped in the middle of a forgotten spot in the Caribbean by providence impoverished,', 'In squalor, grow up to be a hero and a scholar? The ten-dollar founding father without a father']


## 3) Step 3. 데이터 정제  
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!  
preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다. 

In [13]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [16]:
corpus = []

for sentence in raw_corpus: # 
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        

corpus[:10]

['<start> how does a bastard , orphan , son of a whore <end>',
 '<start> and a scotsman , dropped in the middle of a forgotten spot in the caribbean by providence impoverished , <end>',
 '<start> in squalor , grow up to be a hero and a scholar ? the ten dollar founding father without a father <end>',
 '<start> got a lot farther by working a lot harder <end>',
 '<start> by being a lot smarter by being a self starter <end>',
 '<start> by fourteen , they placed him in charge of a trading charter and every day while slaves were being slaughtered and carted away <end>',
 '<start> across the waves , he struggled and kept his guard up <end>',
 '<start> inside , he was longing for something to be a part of <end>',
 '<start> the brother was ready to beg , steal , borrow , or barter then a hurricane came , and devastation reigned <end>',
 '<start> our man saw his future drip , dripping down the drain <end>']

**문장 토큰 개수를 12개로 조정**

In [45]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  

    print(tensor, tokenizer, sep='\n')
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[    2    78   626 ...     3     0     0]
 [   14     6   881 ... 11331     4     3]
 [    9  2040     8 ...     9   635     3]
 ...
 [    2   207     1 ...     0     0     0]
 [    2    76     7 ...     0     0     0]
 [    2    14     6 ...     0     0     0]]


In [46]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력
print(tensor[:5, :10])

[[    2    78   626     9  4376     4  5629     4   611    19]
 [   14     6   881    19     9  1974   808    14     6  4179]
 [    9  2040     8     9  7655    43     6   743  1128 11332]
 [    2    41     9   396  3196   122  1121     9   396  1032]
 [    2   122   543     9   396  7656   122   543     9  1202]]


# \[1]번 보니까 앞에 부분을 잘라서 길이 맞췄다고 볼 수 있음

In [47]:
# 단어 사전이 어떻게 구축되었는지 아래와 같이 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 20: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t
16 : s
17 : that
18 : on
19 : of
20 : .


## 4) Step 4. 평가 데이터셋 분리  
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!   
  
```
enc_train, enc_val, dec_train, dec_val = <코드 작성>

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


out:

Source Train: (124960, 14)
Target Train: (124960, 14)
```


In [48]:
# 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2   78  626    9 4376    4 5629    4  611   19    9 3293    3    0]
[  78  626    9 4376    4 5629    4  611   19    9 3293    3    0    0]


In [49]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

print(type(dataset))
dataset

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [50]:
from sklearn.model_selection import train_test_split

# train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input, 
                                                          test_size=0.2, 
                                                          random_state=20)

In [51]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## 5) Step 5. 인공지능 만들기  
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)  
``` 
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
    
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

```

In [52]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) # Embedding 레이어
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [53]:
# 모델의 최종 출력 텐서 shape
for src_sample, tgt_sample in dataset.take(1): break
print(model(src_sample))

tf.Tensor(
[[[-7.17030707e-05  9.66636726e-05 -1.35926719e-04 ... -5.78529034e-05
   -1.68122162e-04  2.75197181e-05]
  [-3.71309608e-04  1.28697240e-04 -3.36190627e-04 ... -1.08932065e-04
   -3.81891414e-06  7.38477102e-05]
  [-2.65153212e-04  3.11271113e-04 -3.62820545e-04 ... -2.24667689e-04
    3.26630456e-04  1.03909324e-05]
  ...
  [-1.14605750e-03  1.72785786e-03  9.73225688e-04 ... -2.66238785e-04
    2.12322897e-03  1.78654457e-03]
  [-1.00275455e-03  1.84375024e-03  1.07264449e-03 ... -1.51951783e-04
    2.23858887e-03  1.91745337e-03]
  [-8.65304784e-04  1.92454935e-03  1.14755053e-03 ... -6.29854039e-05
    2.33769789e-03  1.99997635e-03]]

 [[ 3.34372926e-05  1.65613717e-04  2.45500032e-05 ... -2.73446258e-05
    1.97154644e-04  6.07340589e-05]
  [ 1.09722467e-04  4.50927415e-04  7.50840845e-05 ... -5.42589441e-05
    4.94283566e-04  1.21947691e-04]
  [ 1.02939288e-04  3.98184784e-04  3.20623687e-04 ... -8.93959514e-05
    6.38325349e-04 -2.10054292e-04]
  ...
  [-3.420221

In [54]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [55]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

# history = model.fit(partial_x_train,
#                     partial_y_train,
#                     epochs=epochs,
#                     batch_size=512,
#                     validation_data=(x_val, y_val),
#                     verbose=1)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=(enc_val, dec_val), verbose=1)

Epoch 1/10
686/686 [==============================] - 99s 144ms/step - loss: 3.6439 - val_loss: 3.2349
Epoch 2/10
686/686 [==============================] - 101s 148ms/step - loss: 3.1296 - val_loss: 2.9753
Epoch 3/10
686/686 [==============================] - 98s 143ms/step - loss: 2.9300 - val_loss: 2.8043
Epoch 4/10
686/686 [==============================] - 101s 147ms/step - loss: 2.7808 - val_loss: 2.6669
Epoch 5/10
686/686 [==============================] - 100s 146ms/step - loss: 2.6526 - val_loss: 2.5399
Epoch 6/10
686/686 [==============================] - 100s 146ms/step - loss: 2.5339 - val_loss: 2.4214
Epoch 7/10
686/686 [==============================] - 100s 145ms/step - loss: 2.4223 - val_loss: 2.3147
Epoch 8/10
686/686 [==============================] - 97s 141ms/step - loss: 2.3179 - val_loss: 2.2038
Epoch 9/10
686/686 [==============================] - 99s 144ms/step - loss: 2.2153 - val_loss: 2.1014
Epoch 10/10
686/686 [==============================] - 100s 146ms/st

In [56]:
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

print(loss)

In [57]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## 6) 잘 만들어졌는지 평가하기

In [58]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True: # 주목!
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [59]:
# init_sentence 를 바꿔가며 해보기 
print(generate_text(model, tokenizer, init_sentence="<start> you"))

print(generate_text(model, tokenizer, init_sentence="<start> we"))

print(generate_text(model, tokenizer, init_sentence="<start> hate"))

print(generate_text(model, tokenizer, init_sentence="<start> baby"))

print(generate_text(model, tokenizer, init_sentence="<start> "))

<start> you know you re right <end> 
<start> we re gonna have to serve it <end> 
<start> hate to see the <unk> <end> 
<start> baby , baby , baby <end> 
<start> i m the one <end> 


## 루브릭

평가문항
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  
   텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?  
->부정적인 단어(hate)에는 \<unk>이 나왔지만, 나머지 문장은 자연스러운 문장을 출력했다.  
   
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  
   특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?  
-> 노드와 다르게 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외해야했다. 그래서 토큰화 과정에 'maxlen = 15'을 추가해 주었다.
  
3. 텍스트 생성모델이 안정적으로 학습되었는가?   
   텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?
-> 마지막 loss가 1.9992 로 2.2보다 낮게 나왔다.  
  
***  
## 고찰  
노드에서 적절한 비유와 함께 설명해서 자연어 과제 중 가장 수월하게 해낼 수 있었다.  
아쉽게도 maxlen을 15로 했음에도 data의 크기가 120000개가 넘었다. 이 점을 더 보완하면 좋을 것 같다. 